In [1]:
from ROOT import TH1F
def sum_histos(histo_list):
    result=TH1F("sum","sum",histo_list[0].GetNbinsX(),0.,1.)
    result.SetDirectory(0)
    for histo in histo_list:
        for i in range (histo.GetNbinsX()):
            sum_=result.GetBinContent(i+1)
            sum_+=histo.GetBinContent(i+1)
            result.SetBinContent(i+1,sum_)
            err_=result.GetBinError(i+1)
            err_+=histo.GetBinError(i+1)
            result.SetBinError(i+1,err_)
    return result

Welcome to JupyROOT 6.22/06


In [2]:
def getall(d):
    "Generator function to recurse into a ROOT file/dir and yield (path, obj) pairs"
    for key in d.GetListOfKeys():
        kname = key.GetName()
        if key.IsFolder():
            # TODO: -> "yield from" in Py3
            for i in getall(d.Get(kname)):
                yield i
        else:
            yield d.Get(kname)

In [3]:
def get_data_by_histo(h):
    histo=h.Clone()
    x=[0]
    y=[0]
    n_events=histo.Integral()
    histo.Scale(1.0/n_events)
    sum_=0.
    dx=1./histo.GetNbinsX()
    for i in range (histo.GetNbinsX()):
        if histo.GetBinContent(i+1) !=0 :
            x.append((i+1)*(dx))
            sum_+=histo.GetBinContent(i+1)
            y.append(sum_)
    if x[len(x)-1]<1 :
        x.append(1.)
        y.append(1.)
    x=np.array(x)
    y=np.array(y)
    return x , y , n_events

In [4]:
from scipy import signal as sig

In [5]:
def butter_filtfilt( x, Wn=0.5, axis=0 ):
    """ butter( 2, Wn ), filtfilt
        axis 0 each col, -1 each row
    """
    b, a = sig.butter( N=2, Wn=Wn )
    return sig.filtfilt( b, a, x, axis=axis, method="gust" )  # twice, forward backward

In [6]:
def ints( x ):
    return x.round().astype(int)

In [7]:
def minavmax( x ):
    return "min av max %.3g %.3g %.3g" % (
        x.min(), 
        x.mean(), 
        x.max() 
    )

In [8]:
def pvec( x ):
    n = len(x) // 25 * 25
    return "%s \n%s \n" % (
        minavmax( x ),
        ints( x[ - n : ]) .reshape( -1, 25 ))

In [9]:
def monofit( a, Wn):
    y=np.log(a)
    """ monotone-increasing curve fit """
    y = np.asarray(y).squeeze() 
    ygrad = np.gradient( y )
    gradsmooth = butter_filtfilt( ygrad, Wn=Wn )  
    ge0 = np.fmax( gradsmooth, 2e-5 )
    ymono = np.cumsum( ge0 )  # integrate, sensitive to first few
    ymono += (y - ymono).mean()
    err = y - ymono
    errstr = "average |y - monofit|: %.2g" % np.abs( err*100 ).mean()
    ymono=np.exp(ymono)
    return ymono, err, errstr

In [10]:
import numpy as np
from scipy.interpolate import PchipInterpolator

def interpolate_data(x,y,step,pot):
    interp_func_mid=PchipInterpolator(x, y)    
    X=np.power(np.arange(0, np.exp(0), step)+step,pot)
    Y=interp_func_mid(X)
    return X,Y


In [11]:
def smooth_interpolation_data(Y,step):
    Wn=step*100
    ymono, err, errstr = monofit( Y , Wn )
    Y=ymono/max(ymono)
    return Y

import numpy as np
from scipy.interpolate import splev, splrep

def get_interp_func(h,step=1e-3,pot=2.1):
    x , y , n_events = get_data_by_histo(h)
    X , Y = interpolate_data(x,y,step,pot)
    Y=smooth_interpolation_data(Y,step)
    spl = interpolate.splrep(X, Y, s=5e-2)
    step = step/5
    xnew = np.arange(0., np.exp(0), step)+step
    ynew = splev(xnew, spl)
    ynew = ynew*n_events
    return PchipInterpolator(xnew, ynew)


    

In [12]:
def get_all_dict(file_path):
    hist_dict={}
    f=TFile(file_path)
    for hh in getall(f):
        hh.SetDirectory(0)
        hist_dict.update({hh.GetName():hh})
    return hist_dict

In [13]:
def get_bins_from_histo(h):
    bins=[h.GetBinContent(bin_+1) for bin_ in range(h.GetNbinsX())]
    return h.GetName() ,  bins

In [14]:
def refine_histo(h):
    n_bins=100
    hist_test=TH1F(h.GetName(),h.GetTitle(),n_bins,0.,1.)
    hist_test.SetDirectory(0)
    interp_func = get_interp_func(h)
    for bin_ in range(hist_test.GetNbinsX()):
        hist_test.SetBinContent(bin_+1,interp_func((bin_+1)/n_bins)-interp_func((bin_)/n_bins))
    hist_test.Scale(h.Integral()/hist_test.Integral())
    return hist_test

In [15]:
def combine_histos(hist_dict,group_dict,refine_names):
    
    final_histos=[]
    for signal in group_dict.keys():
        hist_list=[]
        for keyword in group_dict[signal]:
            for key in hist_dict.keys():
                if keyword in key:
                    hist_list.append(hist_dict[key])
        h=sum_histos(hist_list)
        n_events=h.Integral()
        h.SetName(signal)
        h.SetTitle(signal+";ML-score;nevents(137/fb)")
        if h.GetName() in refine_names:
            h=refine_histo(h)
        final_histos+=[h]
    return final_histos

In [16]:
import os
def conver_histos_to_txt(hist_list,save_dir):
    def write_txt(key):
        path=os.path.join(save_dir,f'{key}.txt')
        with open(path, 'w') as f:
            [f.write(str(bin_)+"\n") for bin_ in bins[key]]
        return path
    bins=dict(map(get_bins_from_histo,hist_list))
    return list(map(write_txt,bins.keys()))


In [17]:
from ROOT import TCanvas
def draw_hist(h,save_dir):
    c1=TCanvas( f'c-{h.GetName()}', '', 0, 0, 1280, 720)
    c1.SetGrid()
    c1.SetLogy()
    h.Draw("HIST")
    c1.SaveAs(os.path.join(save_dir,f"{h.GetName()}.png"))
    del c1

In [18]:
import os
from pathlib import Path
from scipy import interpolate
from ROOT import TFile, TCanvas, TH1F
from multiprocessing import Pool
from multiprocessing import cpu_count as n_cpu


def extract_histos(file):
    parent_dir=os.path.dirname(file)
    save_dir=parent_dir.replace(
        os.path.basename(folder_out),
        os.path.basename(os.getcwd())
    )
    !rm -rf {save_dir}
    !mkdir -p {save_dir}
    #if not("b_tau_tau_semileptonic" in file): continue
    hist_dict=get_all_dict(file)
    group_dict={
        "signal":["LQ"],
        "ttbar": ["ttbar"],
        "single_top": ["stop"],
        "diboson":["ww","wz","zz"],
        "v_jets": ["w_jets","z_jets"]
    }
    refine_names=["diboson","v_jets"]
    final_histos=combine_histos(
        hist_dict,
        group_dict,
        refine_names
    )
    
    conver_histos_to_txt(final_histos,save_dir)
    [draw_hist(histo,save_dir) for histo in final_histos]
    
    f = TFile(
        os.path.join(save_dir,"Logistic_Regresion.root"),
        "RECREATE")
    [h.Write(h.GetName()) for h in final_histos]
    
    return f

folder_out=os.path.join(
    os.path.dirname(os.getcwd()),
    "04_ML_analysis"
)
path_root = Path(folder_out)
files=[root_file.as_posix() for root_file in path_root.glob('**/*.root')]

with Pool(n_cpu()) as p:
    list(p.map(extract_histos,files))

Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Statistical_analysis/MLQ_750/b_b_tau_tau_hadronic/signal.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Statistical_analysis/MLQ_750/b_tau_tau_semileptonic/signal.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Statistical_analysis/MLQ_500/b_tau_tau_semileptonic/signal.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Statistical_analysis/MLQ_1000/b_tau_tau_semileptonic/signal.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Statistical_analysis/MLQ_750/b_b_tau_tau_semileptonic/signal.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/05_Stati